# Quickstart Tutorial for RAGxplorer

In this tutorial, you will learn how to use the `RAGxplorer` package. It will help you visualize your RAG (Retrieval Augmented Generation) embeddings of an external document, such as a PDF, against user queries. Additionally, you will discover how to visualize outputs from various models, including the default `all-MiniLM-L6-v2` embedding model, OpenAI models, and HuggingFace models.

The package performs the following steps to an external document:

1. Loads text from external document
2. An embedding model is selected to vectorize the text into embedding chunks
3. Chroma is used to store embeddings into a vector database
4. [UMAP dimensionality reduction](https://umap-learn.readthedocs.io/en/latest/) is applied to the embeddings
5. 2D projection of embedding chunks is visualized using Plotly, highlighting chunks most relevant to the user query

We will use the [Singapore National AI Strategy](https://file.go.gov.sg/nais2023.pdf) as an example document.

First, let's import `ragxplorer` library.

In [ ]:
import os
import requests
from urllib.parse import urlparse
from typing import Optional

from ragxplorer import RAGxplorer

Next, let's download a pdf to embed and visualize.

In [2]:
pdf_url = "https://file.go.gov.sg/nais2023.pdf"

In [3]:
def download_pdf(pdf_url: str, filename: Optional[str] = None, output_dir: Optional[str] = None) -> str:
    if output_dir is None:
        output_dir = os.getcwd()
    if filename is None:
        filename = os.path.basename(urlparse(pdf_url).path)
    output_path = os.path.join(output_dir, filename)
    response = requests.get(pdf_url)
    response.raise_for_status()
    with open(output_path, "wb") as f:
        f.write(response.content)
    print(f"Downloaded file saved at:\n{output_path}")
    
    return output_path

In [ ]:
pdf_path = download_pdf(pdf_url)

## Embedding with Sentence Transformer Model `all-MiniLM-L6-v2` 

Using this embedding model in `RAGxplorer` does not require an API, whereas using HuggingFace and OpenAI embeddings models does require an API.

For information about this model:
* https://www.sbert.net/
* https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

In [5]:
client = RAGxplorer(embedding_model="all-MiniLM-L6-v2")
client.load_pdf(document_path=pdf_path, verbose=True)
client.visualize_query(
    "How will AI innovation be balanced with ethnical considerations and public trust?"
)

 ~ Building the vector database...
Completed Building Vector Database ✓
 ~ Reducing the dimensionality of embeddings...


100%|██████████| 95/95 [01:21<00:00,  1.16it/s]


Completed reducing dimensionality of embeddings ✓


100%|██████████| 1/1 [00:00<00:00,  1.22it/s]


In the cell above, the `load_pdf` method has the following parameters:
* `document_path`
* `chunk_size` (default 100)
* `chunk_overlap` (default 0)
* `verbose` (default False)
* `umap_params` (default None)


The `visualize_query` method has the following parameters:
* `query`
* `retrieval_method` (`naive` (default) or `HyDE` or `multi_qns` )
* `top_k` (default 5)
* `query_shape_size` (recommended 3 to 10, default 5)
* `import_projection_data` (default None)


## Using OpenAI Embeddings


To use OpenAI embeddings you will need to set your OpenAI API Key token as an environment variable. 

1. Go to https://platform.openai.com/account/api-keys
2. Create a new secret key 
3. Copy your new secret key and paste in below cell and replace `"your-openai-key-here"`

In [6]:
os.environ["OPENAI_API_KEY"] = "your-openai-key-here"

Currently, RAGxplorer supports the following OpenAI embedding models:
* `text-embedding-3-small`
* `text-embedding-3-large`
* `text-embedding-ada-002`


In [7]:
client_openai = RAGxplorer(embedding_model="text-embedding-3-small")
client_openai.load_pdf(
    document_path=pdf_path, 
    chunk_size=200,
    chunk_overlap=50,
    verbose=True)
client_openai.visualize_query(
    query="What measures are being taken to cultivate and attract top AI talent to Singapore?", 
    retrieval_method="HyDE", 
    top_k=6, 
    query_shape_size=10
)

 ~ Building the vector database...
Completed Building Vector Database ✓
 ~ Reducing the dimensionality of embeddings...


100%|██████████| 440/440 [06:22<00:00,  1.15it/s]


Completed reducing dimensionality of embeddings ✓


100%|██████████| 1/1 [00:00<00:00,  1.18it/s]


## Using HuggingFace Embeddings

Similar to OpenAI, to use HuggingFace embeddings you will need to set your HuggingFace API token as an environment variable. 

1. Go to https://huggingface.co/settings/tokens
2. Create a new token 
3. Copy your token key and paste in below cell and replace `"your-huggingface-token-here"`

For more HuggingFace embedding models: https://huggingface.co/models?other=Sentence+Transformers&sort=downloads

In [8]:
os.environ["HF_API_KEY"] = "your-hugginface-token-here"

In [9]:
client_hf = RAGxplorer(embedding_model="thenlper/gte-large")
client_hf.load_pdf(document_path=pdf_path, verbose=True)
client_hf.visualize_query(
    "How will AI innovation be balanced with ethnical considerations and public trust?"
)

 ~ Building the vector database...
Completed Building Vector Database ✓
 ~ Reducing the dimensionality of embeddings...


100%|██████████| 95/95 [01:23<00:00,  1.13it/s]


Completed reducing dimensionality of embeddings ✓


100%|██████████| 1/1 [00:01<00:00,  1.57s/it]


## Conclusion

Thank you for taking the time to read through this tutorial on using `RAGxplorer`. We hope you found it helpful and informative. If you have any questions or feedback, please don't hesitate to reach out. 